In [ ]:
try:
  from googletrans import Translator
except :
  !pip install googletrans　&> /dev/null
  from googletrans import Translator
import re
from requests import get
import json
from google.colab import files

translator = Translator()

In [ ]:
#{ run: "auto" }
url = "" #@param {type:"string"}
url = url.replace('https://github.com/', 'https://raw.githubusercontent.com/').replace('/blob',"")
ipynb = json.loads(get(url).text)
for i, c in enumerate(ipynb['cells']):
    for j, s in enumerate(c['source']):
        if c['cell_type'] == 'markdown':
            sh = re.match("#*\s?", s)
            ipynb['cells'][i]['source'][j] = s[sh.span()[0]:sh.span()[1]] + translator.translate(s[sh.span()[1]:], dest="ja").text + "\n"
        elif c['cell_type'] == 'code':
            for comment in re.findall("#\s*(.+?)#|#\s*(.+)\n?", s):
                comment = ''.join(comment)
                s = s.replace(comment, translator.translate(comment, dest="ja").text)
            ipynb['cells'][i]['source'][j] = s
            
file_name = re.search(".+/(.+?\.ipynb)", url).group(1)
with open(file_name, "w", encoding="utf-8") as f:
    f.write(json.dumps(ipynb))
files.download(file_name)